# The Approach

For the current task, i've used a basic cnn classifier with 2 convolution layers with 32 and 64 filters.

As this is a simple image classification task, A cnn would work the best for a 2d input rather than a simple ML classifier which takes a 1D feature.

## The Data

The data consists of about 78200 training samples in the form of images in each directory of the corresponding labels. 

I loaded the data into a numpy array from png images, i preprocessed them to have smaller values for faster training.

In [ ]:
import keras
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import os
import imageio

# Preprocessing and generating data

In [48]:
# get all labels into a list for further processing
dirlis = ['character_1_ka','character_2_kha','character_3_ga', 'character_4_gha','character_5_kna',
 'character_6_cha','character_7_chha', 'character_8_ja', 'character_9_jha', 'character_10_yna','character_11_taamatar',
 'character_12_thaa','character_13_daa', 'character_14_dhaa', 'character_15_adna', 'character_16_tabala','character_17_tha','character_18_da',
 'character_19_dha','character_20_na','character_21_pa', 'character_22_pha', 'character_23_ba',
 'character_24_bha', 'character_25_ma', 'character_26_yaw', 'character_27_ra', 'character_28_la',
 'character_29_waw', 'character_30_motosaw', 'character_31_petchiryakha', 'character_32_patalosaw',
 'character_33_ha', 'character_34_chhya', 'character_35_tra', 'character_36_gya',
 'digit_0','digit_1', 'digit_2','digit_3', 'digit_4', 'digit_5', 'digit_6', 'digit_7', 'digit_8',
 'digit_9']

# initialising the x and y variables
x_train = []
y_train = []

x_test = []
y_test = []

# defining parameters for the cnn
batch_size = 128
num_classes = 46
epochs = 12

# size of the images
img_rows, img_cols = 32, 32

# iterate over each directory in the dataset and append into x_train and y_train
i = 0
for d in dirlis:
    for each in os.listdir("{}/{}".format("data/training_data/Train", d)):
        x_train.append(imageio.imread("{}/{}/{}".format("data/training_data/Train",d,each)))
        y_train.append(i) 
    i += 1
        
# iterate over each directory in the dataset and append into x_test and y_test
i = 0
for d in dirlis:
    for each in os.listdir("{}/{}".format("data/training_data/Test", d)):
        x_test.append(imageio.imread("{}/{}/{}".format("data/training_data/Test",d,each)))
        y_test.append(i) 
    i += 1

# converting list of numpy arrays to numpy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# as the convolution layer takes channels last we resize it.
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# To nomalise the values so as to have a lighter model that trains faster.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Converting labels to categorical classes in an array of 46 elements where the label respresent's 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Defining model architecture

In [49]:
# model architecture
model = Sequential()

# Used 2 conv2d layers with 32 and 64 filters with a size of 3 X 3
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))

# adding a maxpool layer with a size of 2 X 2
model.add(MaxPooling2D(pool_size=(2, 2)))

# Dropping out 25% of the neurons randomly
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [51]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Training

In [52]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 78200 samples, validate on 13800 samples
Epoch 1/12
78200/78200 [==============================] - 375s 5ms/step - loss: 1.1914 - acc: 0.6643 - val_loss: 0.2467 - val_acc: 0.9347
Epoch 2/12
78200/78200 [==============================] - 371s 5ms/step - loss: 0.4234 - acc: 0.8736 - val_loss: 0.1541 - val_acc: 0.9528
Epoch 3/12
78200/78200 [==============================] - 371s 5ms/step - loss: 0.3036 - acc: 0.9078 - val_loss: 0.1249 - val_acc: 0.9639
Epoch 4/12
78200/78200 [==============================] - 364s 5ms/step - loss: 0.2453 - acc: 0.9245 - val_loss: 0.1026 - val_acc: 0.9701
Epoch 5/12
78200/78200 [==============================] - 362s 5ms/step - loss: 0.2036 - acc: 0.9367 - val_loss: 0.1046 - val_acc: 0.9687
Epoch 6/12
78200/78200 [==============================] - 366s 5ms/step - loss: 0.1773 - acc: 0.9460 - val_loss: 0.0938 - val_acc: 0.9732
Epoch 7/12
78200/78200 [==============================] - 368s 5ms/step - 

# Results

In [53]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.07282351095235398
Test accuracy: 0.9795652173913043


In [54]:
model.save("data/models/final.h5")